In [325]:
import pandas as pd
import numpy as np
# Loading Comma Seperated Data using read_table pandas function in 'reviews_train_data' dataframe
reviews_train_data = pd.read_csv("reviewstrain.txt", delimiter="\t", header=None)
reviews_train_data = pd.DataFrame(reviews_train_data)

In [326]:
reviews_train_data = pd.DataFrame(reviews_train_data[0].str.split(' ',1).tolist(), columns = ['label','reviews_text'])

In [327]:
import numpy as np
def inter(x, y):
    common_tokens = set(y).intersection(x)
    if len(common_tokens) == 0:
        distance = np.nan
    else:
        distance = 1/len(common_tokens)
    return distance

In [328]:
def get_k_neighbours(k, dataframe):#, reviews_train_data):
    dataframe = dataframe.sort_values(by=['distance'])
    top_k = dataframe.head(k)
    top_kth_distance = dataframe[(k-1):].distance.values[0]
    dataframe = dataframe.drop(dataframe.index[0:k])
    dataframe = dataframe[dataframe['distance']==top_kth_distance]
    top_k = top_k.append(dataframe)
    #print(top_k)
    return top_k

In [329]:
def get_predictions(k,reviews_test_data, reviews_train_data):
    #%%time
    i = 0
    predictions = []
    for test_row in reviews_test_data.itertuples():
        test_row_tokens = test_row.reviews_text.split(' ')
        distance = []
        for train_row in reviews_train_data.itertuples():
            train_text = train_row.reviews_text
            train_row_tokens = train_row.reviews_text.split(' ')
            distance.append(inter(train_row_tokens, test_row_tokens))   
        string_match={
            'k_nearest_train_text': reviews_train_data.reviews_text,
            'distance': distance,
            'label': reviews_train_data.label
        }
        string_match = pd.DataFrame(string_match)
        string_match = get_k_neighbours(k,string_match)#,reviews_train_data)
        try:
            predicted_num0 = string_match['label'].value_counts()['0']
        except KeyError:
            predicted_num0 = 0
        try:
            predicted_num1 = string_match['label'].value_counts()['1']
        except KeyError:
            predicted_num1 = 0
        if predicted_num0 > predicted_num1:
            predictions.append(0)
        else:
            predictions.append(1)
        del string_match
        #print('i is : ',i)
        i = i + 1
    predicted_test_data = {
            'testdata_reviews_text': reviews_test_data.reviews_text,
            'actual_label': reviews_test_data.label,
            'predicted_label': predictions
        }
    predicted_test_data = pd.DataFrame(predicted_test_data)
    return predicted_test_data

In [331]:
k_list = [3]
k_df = {}
for k in k_list:
    prediction = pd.DataFrame(columns=['testdata_reviews_text','actual_label','predicted_label'])
    for i in range(5):
        print('i is :::  ',i)
        reviews_test_data1 = reviews_train_data[(i*300):(i*300)+300]
        reviews_train_data1 = reviews_train_data.drop(reviews_train_data.index[(i*300):(i*300)+300])
        print('reviews_test_data  : ',reviews_test_data1) 
        #reviews_test_data1.to_csv('reviews_test_data1'+str(i))
        #reviews_train_data1.to_csv('reviews_train_data1'+str(i))
        print('reviews_train_data  : ',reviews_train_data1)
        print('k is : ',k)
        predicted_test_data = get_predictions(k,reviews_test_data1,reviews_train_data1)
        prediction = prediction.append(predicted_test_data)
    correct_pred = 0
    total_pred = 0
    print('len of prediction : ',len(prediction))
    for row in prediction.itertuples():
        total_pred = total_pred + 1
        if int(row.actual_label) == int(row.predicted_label):
            correct_pred =correct_pred + 1
    print('correct_pred :',correct_pred)
    print('total_pred :',total_pred)
    accuracy = (correct_pred/total_pred)*100
    print('accuracy : ',accuracy)
    print(total_pred)  
    k_df.update({k : accuracy})
print(k_df)    

i is :::   0
reviews_test_data  :      label                                       reviews_text
0       1  a stirring , funny and finally transporting re...
1       1  a real winner -- smart , funny , subtle , and ...
2       0  a dim-witted and lazy spin-off of the animal p...
3       1  ` anyone with a passion for cinema , and indee...
4       0  a crass and insulting homage to great films li...
5       0                              an infuriating film .
6       0  stealing harvard will dip into your wallet , s...
7       1  a poignant lyricism runs through balzac and th...
8       0  it 's always disappointing when a documentary ...
9       1  there are scenes of cinematic perfection that ...
10      0  what ensues are much blood-splattering , mass ...
11      1                             an impressive hybrid .
12      0  the script 's judgment and sense of weight is ...
13      0  it ends up being neither , and fails at both e...
14      1                    it is definitely wort

In [313]:
max_key = max(k_df, key=lambda k: k_df[k])

In [314]:
max_key

3

In [315]:
reviews_test_data2 = pd.read_csv("reviewstest.txt", delimiter="\t", header=None)
reviews_test_data2 = pd.DataFrame(reviews_test_data2)
reviews_test_data2 = pd.DataFrame(reviews_test_data2[0].str.split(' ',1).tolist(), columns = ['label','reviews_text'])
reviews_test_data2


,label,reviews_text
0,1,"funny , sexy , devastating and incurably roman..."
1,1,cool gadgets and creatures keep this fresh .
2,1,"fathers and sons , and the uneasy bonds betwee..."
3,0,after that it becomes long and tedious like a ...
4,0,what should have been a cutting hollywood sati...
5,0,nothing about the film -- with the possible ex...
6,1,"the overall result is an intelligent , realist..."
7,1,`` red dragon '' is entertaining .
8,0,"despite all the closed-door hanky-panky , the ..."
9,1,a genuinely funny ensemble comedy that also as...


In [316]:
prediction_k = pd.DataFrame(columns=['testdata_reviews_text','actual_label','predicted_label'])
predicted_test_data_k = get_predictions(max_key,reviews_test_data2,reviews_train_data)
prediction_k = prediction_k.append(predicted_test_data_k)

In [317]:
prediction_k

,testdata_reviews_text,actual_label,predicted_label
0,"funny , sexy , devastating and incurably roman...",1,1
1,cool gadgets and creatures keep this fresh .,1,0
2,"fathers and sons , and the uneasy bonds betwee...",1,1
3,after that it becomes long and tedious like a ...,0,1
4,what should have been a cutting hollywood sati...,0,0
5,nothing about the film -- with the possible ex...,0,0
6,"the overall result is an intelligent , realist...",1,1
7,`` red dragon '' is entertaining .,1,1
8,"despite all the closed-door hanky-panky , the ...",0,0
9,a genuinely funny ensemble comedy that also as...,1,1


In [318]:
total_pred_k = 0
correct_pred_k = 0
false_negative = 0
false_positive = 0
for row in prediction_k.itertuples():
        total_pred_k = total_pred_k + 1
        if int(row.actual_label) == int(row.predicted_label):
            correct_pred_k =correct_pred_k + 1
        elif  (int(row.actual_label) == 1 and int(row.predicted_label) == 0):   
            false_negative = false_negative + 1
        elif  (int(row.actual_label) == 0 and int(row.predicted_label) == 1):   
            false_positive = false_positive + 1     
print('correct_pred_k :',correct_pred_k)
accuracy_k = (correct_pred_k/total_pred_k)*100
print('accuracy_k : ',accuracy_k)
print('false_negative : ',false_negative)
print('false_positive : ',false_positive)
print(total_pred_k)

correct_pred_k : 295
accuracy_k :  59.0
false_negative :  61
false_positive :  144
500
